In [1]:
from src.utils import read_json, write_json

train = read_json(r"data\smartui\train.json")
val = read_json(r"data\smartui\val.json")
test = read_json(r"data\smartui\test.json")

In [2]:
def get_label_set(*lists):
    label_set = {}
    count = 0
    for current_lst in lists:
        for dict in current_lst:
            for labels in dict['label']:
                if labels not in label_set:
                    label_set[labels] = count 
                    count += 1
    return label_set

In [3]:
from src.dataset import SentenceLabelDataset

label_set = get_label_set(train, test)
train_set = SentenceLabelDataset(train, labelSet=label_set)

In [9]:
label_set

{'Work At Home Request': 0,
 'oos': 1,
 'Firewall Rules Setup': 2,
 'New Virtual Machine Request': 3,
 'Mobile Email Access': 4,
 'Desktop Software Installation Support': 5,
 'Working Room Setup': 6}

In [10]:
train_set[15]

{'text': "Hello! I'm having some difficulty completing a form for a new virtual machine request. Can you lend me a hand?",
 'class': ['New Virtual Machine Request'],
 'input_ids': [101,
  7592,
  999,
  1045,
  1005,
  1049,
  2383,
  2070,
  7669,
  7678,
  1037,
  2433,
  2005,
  1037,
  2047,
  7484,
  3698,
  5227,
  1012,
  2064,
  2017,
  18496,
  2033,
  1037,
  2192,
  1029,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'label': [3]}